# Lesson 3: Building an Agent Reasoning Loop

![alt text](image1.png)

![alt text](image2.png)

![alt text](image3.png)

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [6]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) outlining goals, user stories, and competitive analysis. The Architect designs the system based on the PRD, focusing on technical specifications and system architecture diagrams. The Project Manager breaks down the project into tasks and assigns them to Engineers. Engineers then develop the code based on the assigned tasks. The QA Engineer is responsible for generating unit test code and reviewing it to ensure high-quality software.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication between agent rol

In [7]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-15
last_modified_date: 2024-04-23

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [8]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and Humaneval-X.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and Humaneval-X.


In [9]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.


## Lower-Level: Debuggability and Control

In [10]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [11]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [12]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and formulating a detailed Product Requirement Document (PRD). The Architect translates requirements into system design components. The Project Manager distributes tasks based on the system design, while Engineers execute designated classes and functions. The QA Engineer formulates test cases to ensure code quality. Each role has specific responsibilities tailored to different aspects of the collaborative software development process within the MetaGPT framework.


In [13]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 81989a98-128f-467a-9734-f3a5729008d4: 1
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and formulating a detailed Product Requirement Document (PRD). The Architect translates requirements into system design components. The Project Manager distributes tasks based on the system design, while Engineers execute designated classes and functions. The QA Engineer formulates test cases to ensure code quality. Each role has specific responsibilities tailored to different aspects of the collaborative software development process within the MetaGPT framework.


In [14]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 81989a98-128f-467a-9734-f3a5729008d4: 1


TaskStep(task_id='81989a98-128f-467a-9734-f3a5729008d4', step_id='5679c63e-0074-44b4-9959-d8921a8bda7e', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [15]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a subscription mechanism. The shared message pool allows agents to exchange messages directly, while the subscription mechanism enables agents to select and follow information based on their role profiles. This approach ensures efficient communication, transparency in message exchange, and relevance in the information received by each agent. Additionally, agents review previous feedback before each project to adjust their constraint prompts, continuously learning from past experiences and improving the overall multi-agent system. The structured workflow involves different roles like Product Manager, Architect, Project Manager, Engineers, and QA Engi

In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a subscription mechanism. The shared message pool allows agents to exchange messages directly, while the subscription mechanism enables agents to select and follow information based on their role profiles. This approach ensures efficient communication, transparency in message exchange, and relevance in the information received by each agent. Additionally, agents review previous feedback before each project to adjust their constraint prompts, continuously learning from past experiences and improving the overall multi-agent system. The structured workflow involves different roles like Product Manager, Architect, Project Manager, Engineers, and QA Engineer, ensuring that information is effectively shared among the agents involved in the collaborative tasks.
True


In [17]:
response = agent.finalize_response(task.task_id)

In [18]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a subscription mechanism. The shared message pool allows agents to exchange messages directly, while the subscription mechanism enables agents to select and follow information based on their role profiles. This approach ensures efficient communication, transparency in message exchange, and relevance in the information received by each agent. Additionally, agents review previous feedback before each project to adjust their constraint prompts, continuously learning from past experiences and improving the overall multi-agent system. The structured workflow involves different roles like Product Manager, Architect, Project Manager, Engineers, and QA Engineer, ensuring that information is effectively shared among the agents involved in the collaborative tasks.
